In [1]:
import torch
import numpy as np

In [16]:
def generate_class_mask(label, classes):
    label, classes = torch.broadcast_tensors(label,
                                             classes.unsqueeze(1).unsqueeze(2))
    print(classes)
    print(label)
    class_mask = label.eq(classes)
    print(class_mask)
    class_mask = class_mask.sum(0, keepdims=True)
    print(class_mask)
    return class_mask

def get_class_masks(labels, rcm=None):
    class_masks = []
    if rcm != None:
        rcm = torch.tensor(rcm)
    for label in labels:
        classes = torch.unique(labels)
        nclasses = classes.shape[0]
        class_choice = np.random.choice(
            nclasses, int((nclasses + nclasses % 2) / 2), replace=False)
        classes = classes[torch.Tensor(class_choice).long()]
        if rcm != None:
            classes = torch.cat((classes, rcm))
            classes = torch.unique(classes)
        class_masks.append(generate_class_mask(label, classes).unsqueeze(0))
    return class_masks

In [17]:
rcm = [7,8,9]
x = torch.randint(10, (2,1,4,4))
print(x)
y = get_class_masks(x, rcm=rcm)
print(y)

tensor([[[[1, 1, 8, 1],
          [2, 4, 3, 7],
          [5, 2, 6, 5],
          [8, 5, 6, 8]]],


        [[[1, 0, 6, 8],
          [9, 9, 9, 5],
          [2, 8, 4, 7],
          [4, 4, 6, 2]]]])
tensor([[[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]],

        [[2, 2, 2, 2],
         [2, 2, 2, 2],
         [2, 2, 2, 2],
         [2, 2, 2, 2]],

        [[3, 3, 3, 3],
         [3, 3, 3, 3],
         [3, 3, 3, 3],
         [3, 3, 3, 3]],

        [[6, 6, 6, 6],
         [6, 6, 6, 6],
         [6, 6, 6, 6],
         [6, 6, 6, 6]],

        [[7, 7, 7, 7],
         [7, 7, 7, 7],
         [7, 7, 7, 7],
         [7, 7, 7, 7]],

        [[8, 8, 8, 8],
         [8, 8, 8, 8],
         [8, 8, 8, 8],
         [8, 8, 8, 8]],

        [[9, 9, 9, 9],
         [9, 9, 9, 9],
         [9, 9, 9, 9],
         [9, 9, 9, 9]]])
tensor([[[ True,  True, False,  True],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, Fa

In [1]:
import torch
x = torch.randint(10, (2,3,4,4))
q = torch.rand(2,1,4,4)
y = x * q
print(y.shape)

torch.Size([2, 3, 4, 4])


In [44]:
import torch
q = torch.rand(2,1,4,4)
x = torch.rand(2,3,4,4)
print(q)

q = q.permute(1,0,2,3)
mask = torch.rand(2,1,4,4)
mask = mask > 0.8
mask = mask.squeeze(1)
print(mask)
q = q[:,mask]
print(q.shape)
print(q)
q = q.permute(1,0)
q = torch.softmax(q,dim=0)
print(q)
x = x.permute(1,0,2,3)
x = x[:,mask]
x = x.permute(1,0)
print(x.shape)
print(x)
proto = torch.sum(x * q, dim=0)
print(proto.shape)
print(proto)

tensor([[[[0.1583, 0.8226, 0.5751, 0.3185],
          [0.4028, 0.8975, 0.0347, 0.8184],
          [0.5945, 0.0298, 0.1822, 0.9132],
          [0.0612, 0.2392, 0.5988, 0.4100]]],


        [[[0.0520, 0.2864, 0.4678, 0.4094],
          [0.4137, 0.9373, 0.0261, 0.2928],
          [0.1457, 0.4506, 0.7801, 0.0441],
          [0.0479, 0.5041, 0.8837, 0.4835]]]])
tensor([[[ True, False, False, False],
         [False, False, False, False],
         [False, False,  True, False],
         [False, False, False, False]],

        [[False, False, False,  True],
         [False, False, False, False],
         [ True, False, False, False],
         [False, False,  True, False]]])
torch.Size([1, 5])
tensor([[0.1583, 0.1822, 0.4094, 0.1457, 0.8837]])
tensor([[0.1572],
        [0.1610],
        [0.2020],
        [0.1552],
        [0.3246]])
torch.Size([5, 3])
tensor([[0.7169, 0.4231, 0.0713],
        [0.8637, 0.3308, 0.6710],
        [0.9922, 0.2924, 0.5867],
        [0.0264, 0.6836, 0.0332],
        [

In [6]:
import torch
x = torch.rand(3,2)
x[1,:]=0
print(x)
y = torch.rand(3,2)
print(y)
mask = torch.sum(x,dim=1)==0
print(mask)
x[mask,:]=y[mask,:]
print(x)


tensor([[0.0224, 0.4038],
        [0.0000, 0.0000],
        [0.6974, 0.6937]])
tensor([[0.3322, 0.0603],
        [0.8655, 0.6816],
        [0.0124, 0.1320]])
tensor([False,  True, False])
tensor([[0.0224, 0.4038],
        [0.8655, 0.6816],
        [0.6974, 0.6937]])
